<a href="https://colab.research.google.com/github/Gach-omba/nlp/blob/main/t5%2Bparrot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
!pip install git+https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git to /tmp/pip-req-build-etyk8y69
  Running command git clone --filter=blob:none --quiet https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git /tmp/pip-req-build-etyk8y69
  Resolved https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git to commit 720a87a1ee557d8ed8d9a021adbdd1dd5616c5f9
  Preparing metadata (setup.py) ... done


In [41]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [42]:
import torch
from transformers import AutoTokenizer , AutoModelWithLMHead

In [43]:
tokenizer=AutoTokenizer.from_pretrained('t5-base')
model=AutoModelWithLMHead.from_pretrained('t5-base')

In [44]:
from parrot import Parrot
import torch
import warnings
warnings.filterwarnings("ignore")


def random_state(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

random_state(1234)


#Init models (make sure you init ONLY once if you integrate this to your code)
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5")

In [45]:
def break_sentences(string):
    # Define common sentence delimiters
    delimiters = ['. ', '? ', '! ']

    # Remove leading and trailing whitespaces
    string = string.strip()

    # Break down the string into individual sentences
    sentences = []
    current_sentence = ''
    for char in string:
        current_sentence += char
        if any(current_sentence.endswith(delimiter) for delimiter in delimiters):
            sentences.append(current_sentence.strip())
            current_sentence = ''

    # Add the remaining sentence if any
    if current_sentence:
        sentences.append(current_sentence.strip())

    return sentences


In [ ]:
phrases=break_sentences("""Once upon a time, in a small village nestled deep within a lush forest, there lived a young girl named Lily. She had a heart full of curiosity and a spirit that yearned for adventure. Despite the simplicity of her village life, Lily craved something more.

One bright morning, while the village was still asleep, Lily made a decision that would change her life forever. She packed a small bag with provisions and set off on a journey into the unknown. With each step, she felt the thrill of the unknown tingling in her veins.

As she ventured deeper into the forest, the trees whispered secrets in hushed tones. The birds above sang songs of encouragement, urging her forward. Guided by an invisible force, Lily found herself stumbling upon a hidden pathway that led to a forgotten world.

The hidden world was breathtakingly beautiful, filled with vibrant colors and magical creatures. It was a place where dreams came to life and impossibilities became possibilities. With wide-eyed wonder, Lily explored the enchanted forest, encountering talking animals and friendly forest sprites along the way.

One day, as she wandered deeper into the forest, Lily stumbled upon an ancient and mysterious book. Its pages were filled with cryptic symbols and unknown languages. Intrigued, Lily decided to take the book with her, hoping to unravel its secrets.

As days turned into weeks, Lily poured over the book, deciphering its enigmatic contents. With each revelation, her understanding of magic grew. She discovered spells to heal the wounded, charms to bring joy to the sorrowful, and incantations to protect the innocent.

Word of Lily's newfound powers spread throughout the hidden world, reaching the ears of the forest's most feared inhabitant, the wicked sorceress Morgana. Consumed by jealousy and a desire for absolute power, Morgana set out to capture Lily and steal her magical abilities.

Lily soon found herself in a battle against darkness, facing Morgana's malevolent forces. With the help of her newfound friends, the talking animals and forest sprites, Lily stood her ground. She summoned her courage and tapped into the vast well of magic within her, casting powerful spells that illuminated the forest with hope.

In an epic showdown, Lily confronted Morgana, their powers clashing like thunder in the night. It was a battle of light against darkness, good against evil. In the end, Lily's pure heart and unwavering belief in the power of love triumphed over Morgana's darkness.

With Morgana defeated, the hidden world rejoiced, celebrating Lily as their savior. Lily, forever changed by her extraordinary journey, became a guardian of the enchanted forest, using her magic to bring harmony and peace to all who resided there.

And so, the tale of Lily, the brave young girl who discovered her own magic, echoed through the generations, inspiring others to embrace their inner power and seek adventure in the world around them.""")
results = []  # Initialize an empty list to store the results

for phrase in phrases:
    print("-" * 100)
    print("Input_phrase: ", phrase)
    print("-" * 100)
    try:
        para_phrases = parrot.augment(input_phrase=phrase,
                                      use_gpu=False,
                                      do_diverse=True,
                                      adequacy_threshold=0.80,
                                      fluency_threshold=0.80)
        for para_phrase in para_phrases:
            modified_phrase = para_phrase[0]  # Get the first phrase (extracted from tuple)
            
            # Add a period at the end of each sentence
            modified_phrase = modified_phrase.rstrip()  # Remove trailing whitespaces
            if modified_phrase and not modified_phrase.endswith('.'):
                modified_phrase += '.'
            
            # Capitalize the starting letter of every sentence
            modified_phrase = '. '.join(sentence.capitalize() for sentence in modified_phrase.split('. '))
            
            results.append(modified_phrase)  # Append the modified phrase to the results list
            break  # Break the loop after processing the first phrase
    except:
        print("No paraphrases returned")

combined_string = ' '.join(results)  # Combine the phrases into a single string with spaces between them

print(combined_string)  # Print the modified combined string




In [53]:
print(combined_string)

In a small village deep in a lush forest once lived a young girl named lily. In her heart she had curiosity and a desire for adventure. On a bright morning while the village was still asleep lily made a decision that would change her life forever despite the simplicity of her village life. She packed a small bag with provisions and set off for the unknown. As she ventured deeper into the woods the trees whispered secrets in hushed tones. The birds above sang songs of encouragement urging her to move on. With an invisible force lily found herself searching for a hidden path which led to a forgotten world the hidden world was breathtakingly beautiful filled with vibrant colors and magical creatures. Is it a place where dreams come to life and where impossibility becomes possibility?. When lily explored the enchanted forest with wide-eyed wonder she encountered talking animals and friendly forest sprites along the way one day as she wandered deeper into the forest lily stumbled upon an an

In [51]:
chunk_size = 400
chunks = [combined_string[i:i+chunk_size] for i in range(0, len(combined_string), chunk_size)]

In [52]:
summaries = []
for chunk in chunks:
    inputs = tokenizer.encode("summarize: " + chunk, return_tensors='pt', max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(outputs[0])
    summaries.append(summary)

# Concatenate the summaries
full_summary = ' '.join(summaries)
print(full_summary)


<pad> a village girl named lily made a decision that would change her life forever despite the simplicity of her village life. she packed a small bag with provisions and set off for the unknown. as she ventured deeper into the woods lily's curiosity and a desire for adventure changed her life forever.</s> <pad> lily found herself searching for a hidden path which led to a forgotten world. with an invisible force lily found herself searching for a hidden path. the hidden world was breathtakingly beautiful filled with vibrant colors and magical creatures.</s> <pad> ly explored the enchanted forest with wide-eyed wonder. one day as she wandered deeper into the forest lily stumbled upon an ancient and mysterious book. the pages were filled with cryptic symbols and unknown languages. lily decided to take the book with her, hoping to unravel its secrets.</s> <pad> ly discovered spells to heal the wounded, charms to bring joy to the sorrowful. word of ly's newfound powers spread throughout th